In [1]:
import numpy as np
import torch
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from model import AttentionModel
from model_PCA_correlation import AttentionModel_PCA
from dcascore import *
from utils import read_fasta_alignment, remove_duplicate_sequences, add_PCA_coords

# back to original path (in PLM)
sys.path.pop(0)  # Removes the parent_dir from sys.path
from model import AttentionModel

from plm_gen_methods import generate_plm_n_save, generate_coords_n_save, generate_multiple_targets_n_save
from seq_utils import read_tensor_from_txt, set_seed, letters_to_nums, modify_seq 


In [2]:
print(current_dir)
print(parent_dir)

/Users/marzioformica/Desktop/EPFL/Master/StageLBS/PCA_gen_AI/CODE/AttentionDCA_python/src/PLM
/Users/marzioformica/Desktop/EPFL/Master/StageLBS/PCA_gen_AI/CODE/AttentionDCA_python/src


In [3]:
"""
    Load Q, K, V matrices from jdoms (after training)
"""
set_seed()
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]


In [4]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)

torch.Size([64, 65, 65])
34
65


In [5]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))

##############################################################


In [3]:
"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp)


Total sequences read: 14502
Sequences after filtering: 14502
(63, 14502)
Removing duplicate sequences...
Done: 14502 -> 14502 sequences after removing duplicates.
[17  5]


In [7]:
"""
    Generate sequences with PLM initialization from a sequence different betas
"""
# init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
# init_sequence_num = letters_to_nums(init_sequence)
# ratio = 0.1
# init_sequence_num = modify_seq(init_sequence_num, ratio)

N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]
for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"

    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp, beta=b)
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=b)

100%|██████████| 15000/15000 [04:39<00:00, 53.71it/s]


Generated sequences (letters): ['W--KYLGLTGPV-LAELKLSWRRLISVWHPDRFGGQRVTEDFLQRINVANSTLKQ--------', 'W--KYLGLTGPV-LAELKLSWRRLISVWHPDRFGGQRVTEDFLQRINVANSTLKQ---T----', 'W--PYLGLTGPV-LAELKLSWRRLISVWHPDRFGGQRVTEDFLQRINVANSTLKQ---T----', 'W--PYLGLTGPV-LAELKLSWRRLLSVWHPDRFGGQRVTEDFLQRINVANSTLKQ---T----', 'W--PYLGLTGPV-LAELKLSWRRLLSVWHPDRFGGQRVTEDFLQRINVAPSTLKQ---T----']
Generated sequences saved to generated_sequencesPCA_comp


100%|██████████| 15000/15000 [04:37<00:00, 53.97it/s]


Generated sequences (letters): ['PEAYFSAYGNRSMTARQVSVNSLYRCMWAWQWVDDTTCMEWHSGAIEWVDMVQ-AAHIHFHER', 'PEAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMEWHSGAIEWVDMVQ-AAHIHFHER', 'PPAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMEWHSGAIEWVDMVQ-AAHIHFHER', 'PPAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMEWHSGAIEWVDMVQ-AAHIKFHER', 'PPAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMIWHSGAIEWVDMVQ-AAHIKFHER']
Generated sequences saved to generated_sequencesPCA_comp


100%|██████████| 15000/15000 [04:37<00:00, 54.04it/s]


...

Generated sequences (letters): ['DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLSDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLSDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLSDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLRDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLRDQNRRT---']
Generated sequences saved to generated_sequencesPCA_comp


100%|██████████| 15000/15000 [04:38<00:00, 53.94it/s]


Generated sequences (letters): ['DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEISEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD']
Generated sequences saved to generated_sequencesPCA_comp


  0%|          | 12/15000 [00:00<04:41, 53.16it/s]c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:52: RuntimeWarning: overflow encountered in exp
  prob = np.exp(self.beta * sum_energy)  # unnormalized
c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:63: RuntimeWarning: invalid value encountered in divide
  probs /= probs.sum()
  0%|          | 16/15000 [00:00<05:00, 49.90it/s]


ValueError: probabilities contain NaN

## MODEL 2

In [2]:
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]

In [3]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens.shape)
print(Jtens.shape[-1])

torch.Size([64, 63, 63])
21
63
torch.Size([21, 21, 63, 63])
63


In [4]:
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N1=Q_1.shape
_,_,N2=K_1.shape
_,q1,q2=V_1.shape
model=AttentionModel_PCA(H,d,N1,N2,q1,q2,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

# i_indices = torch.arange(L, device=device).unsqueeze(1)
# j_indices = torch.arange(L, device=device).unsqueeze(0)
# mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
# W = W * mask
    
# Compute Jtens
Jtens_PCA = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens_PCA.shape)

torch.Size([64, 63, 2])
21
63
torch.Size([21, 35, 63, 2])


In [5]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences_2Model_PCA"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]),J_PCA=Jtens_PCA)
N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]

for b in betas:
    save_name = f"gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1"
    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,J_PCA=Jtens_PCA,PCA_comp_list=np.array([23,23]))
    if b!=1:
        save_name = f"gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}"
        generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=b,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA,beta_PCA=b,PCA_comp_list=np.array([23,23]))

##############################################################

"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA)

"""
    Generate sequences with PLM initialization from a sequence different betas
"""
# init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
# init_sequence_num = letters_to_nums(init_sequence)
# ratio = 0.1
# init_sequence_num = modify_seq(init_sequence_num, ratio)

N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4,10]

for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"
    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp, beta=b,J_PCA=Jtens_PCA)
    if b!=1:
        save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b_1_b_PCA{b}"
        generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=b,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA,beta_PCA=b)

100%|██████████| 15000/15000 [07:53<00:00, 31.70it/s]


Generated sequences (letters): ['PSAEEILYWFHPCHIRVGQKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VSFVMCCIL', 'PSAEEILYWFHPCHIRVGQKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFVMCCIL', 'PSAEEILYWFHPCHIRVGDKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFVMCCIL', 'PSAEEILYWFHPCHRRVGDKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFVMCCIL', 'PSAEEILYWFHPRHRRVGDKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFVMCCIL']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [07:48<00:00, 32.02it/s]


Generated sequences (letters): ['HSIPWGWKSYEWETFEWTFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGSETPG-RCGIKWLM', 'HSIPWGWKSYEWETFEWKFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGSETPG-RCGIKWLM', 'HSIPWGWCSYEWETFEWKFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGSETPG-RCGIKWLM', 'HSIPWGWCSYEWETFEWKFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGQETPG-RCGIKWLM', 'HSIPWGWCSYEWETFEWKFELDWSGPM-VNSEWYTYFRHWNFWMPAFWGQETPG-RCGIKWLM']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [07:47<00:00, 32.08it/s]


...

Generated sequences (letters): ['---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [08:05<00:00, 30.87it/s]


Generated sequences (letters): ['---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [08:08<00:00, 30.72it/s]


Generated sequences (letters): ['---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------', '---------------------------------------------------------------']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


## ADDED - NEW MODEL

In [10]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences_2Model_PCA"
N_seqs = 15000
save_name = "generated_sequences_randinit_30000_25_6"
PCA_comp_list=np.array([25,6])
generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=PCA_comp_list,J_PCA=Jtens_PCA)

print("done generating sequences with PLM initialization from a random sequence")


100%|██████████| 15000/15000 [03:19<00:00, 75.22it/s]


Generated sequences (letters): ['N-K-MSKIYDWSKRATYRAVTMLMQFCSGPHTECDIHWDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWSKRATYRAVTMLMQFCSGPHTECDIHWDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWSKRATYRAVTMLMQFCSGPHTECDI-WDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWSKRATYRAVTALMQFCSGPHTECDI-WDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWS-RATYRAVTALMQFCSGPHTECDI-WDFNNKNESADQLC-NHKT-VCHQLF']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp
done generating sequences with PLM initialization from a random sequence


In [ ]:
##############################################################

"""
    Generate sequences with PLM initialization from a sequence
"""
import platform

if platform.system() == "Darwin":  # macOS
    base_path = '/Users/marzioformica/Desktop/EPFL/Master/StageLBS/PCA_gen_AI/CODE/DataAttentionDCA/jdoms'
else:  # assume Windows for Youss
    base_path = r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms"

file_test_data = f"{base_path}/jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
print("initial sequence number:", init_sequence_num)
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=15000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_20_20"

generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp,PCA_comp_list=PCA_comp_list, J_PCA=Jtens_PCA)
print("done generating sequences with PLM initialization from an initial sequence sequence")


Total sequences read: 14502
Sequences after filtering: 14502
(63, 14502)
Removing duplicate sequences...
Done: 14502 -> 14502 sequences after removing duplicates.
[17  5]
initial sequence number: [11 12 19  3 17  9  5  7  8  3  5  0 15 13  2  3  7  8  8  0 19 14  3  9
  0  8  8 19  6 12  2 13 19  5  2 11 12  9  0  3  3  8 10 14  3  9 11  3
  0 19  2 19  9 15  8 11 15 20 20 20 20 20 20 17  5]


100%|██████████| 15000/15000 [03:31<00:00, 70.87it/s]


Generated sequences (letters): ['NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDLRHFYDYRSKAS--HG--', 'NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDLRHFYDYRSKAS--HG--', 'NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDIRHFYDYRSKAS--HG--', 'NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDIRHFYDYRSKAS--HG--', '-WYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDIRHFYDYRSKAS--HG--']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp
done generating sequences with PLM initialization from an initial sequence sequence


## Generate from multiple coords

In [7]:
save_dir = "generated_sequences_2Model_PCA_multiple"
N_seqs = 10000
save_name_prefix = f'generated_sequences_randinit_{N_seqs}_mult'
# take 10 random coordinates from the PCA space (0-34)
target_coords_list = [(np.random.randint(0, 35), np.random.randint(0, 35)) for _ in range(10)]
generate_multiple_targets_n_save(save_dir, save_name_prefix, J=Jtens, target_coords_list=target_coords_list, N_seqs=N_seqs, init_sequence=None, beta=1, nb_PCA_comp=0, J_PCA=Jtens_PCA, beta_PCA=1)

Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [03:38<00:00, 45.82it/s]


[0] Generated sequences for target (17, 15) (first 5): ['NSNMCQTRGYQVGNQLVGTVMGGALQIWPADGCERWQEM-SYTG-IIT--M-NPVVCSTGTDM', 'NSNMCQTRGYQVGNALVGTVMGGALQIWPADGCERWQEM-SYTG-IIT--M-NPVVCSTGTDM', 'NSNMCQTRGYQVGNALVGTVMGGALQIWPADGCERWQEM-SYTG-IIT-YM-NPVVCSTGTDM', 'NSN-CQTRGYQVGNALVGTVMGGALQIWPADGCERWQEM-SYTG-IIT-YM-NPVVCSTGTDM', 'NSN-CQTRGYQVGNVLVGTVMGGALQIWPADGCERWQEM-SYTG-IIT-YM-NPVVCSTGTDM']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [03:41<00:00, 45.09it/s]


[1] Generated sequences for target (20, 34) (first 5): ['MILWMSLGEMHKPFIHV-TA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCQVDC-IAGIYNA', 'MILWMSLGEMHKPFIHV-TA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCQVDCLIAGIYNA', 'MILWMSLGEMHKPFIHVDTA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCQVDCLIAGIYNA', 'MILWMSLGEMHKPFIHVDTA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCRVDCLIAGIYNA', 'MILWMSLGEMHKPFIHVDTA-SRYCAPDIMIRHDWRMDPALGPERMGFDHCRVDCLIAGIYNA']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


...

[7] Generated sequences for target (31, 27) (first 5): ['FHKYVAQQRAHTRASWRCLDRTGMLEAA-GSLPEFSHISVPIFEQWYIYPVMYPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRTGMLEAA-GSLPEFSHISVPIFEQWYIYPVMHPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRSGMLEAA-GSLPEFSHISVPIFEQWYIYPVMHPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRKGMLEAA-GSLPEFSHISVPIFEQWYIYPVMHPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRKGMLEAA-GSLPEFRHISVPIFEQWYIYPVMHPNFG-SGVKY']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [04:14<00:00, 39.25it/s]


[8] Generated sequences for target (33, 21) (first 5): ['YPMNFILFQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQGEC-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQGEC-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQYEC-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQYEA-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY-VTEPHEQYEA-SNS']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [04:08<00:00, 40.27it/s]


[9] Generated sequences for target (1, 1) (first 5): ['MIAHF-YFDIPTTTFRYGKPAWIEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWIEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWIEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWGEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWGEETILFGNDMPARWRI-PDHWC-WEYRYVQSIHAWYQRFI']
